<h1><center> Project: Using Machine Learning Methods to Predict Premier League Games <center></h1>

One of the best parts of being a data scientist is being able to apply the skills you know to the hobbies you love. I enjoy watching soccer, a particulary difficult sport to when attempting to predict outcomes (FiveThirtyEight, which I compare my model against at the end, only achieves about 50% accuracy, [according to this curious fan's analysis](https://www.reddit.com/r/USLPRO/comments/dl316s/just_how_accurate_is_538/).

I aim to achieve an accuracy rating within 5% of this 50% threshold through the following process:
- web scrape Premier League game results from the official organization's website
- create two models: one purely statistical and one involving logistic regression
- fit and test models, report general accuracy
- for the first matchweek of the 2022/23 season, predict outcomes with models and compare with FiveThirtyEight's results

Throughout each step, I'll explain the reasoning behind my methods. At the end, a brief discussion of results will follow, as well as ways this project can be improved upon in the future.

### Environment Preparation

We'll be using BeautifulSoup for our webscraping portion of our project and scikit-learn for our logistic regression model.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import csv
import time
import json
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_val_predict

### Web Scraping: All Seasons from 1992/93 - 2021/22

As opposed to getting a nice clean dataset from Kaggle, I'm choosing to go with a slightly more rigorous path. Results for each season are posted on the Premier League's official website. Using the code below, I target separate pages for different matchweeks of each season, obtaining the following information:
- Team
- Opponent
- Team Score
- Opponent Score
- Score Difference
- Bookings (Yellow and Red)
- Outcome

It is important to note that not all of this information will necessarily be used. However, I'm setting up information for further analysis if I choose to expand on this project in the future.

![title](premier_webpage.png)

<i><center>*Figure 1: an example webpage of results from the official Premier League website*<center></i>

In [67]:
# id list of all premier league matchdays for all seasons from 1992/93 to 2021/22
matchday_ids = list(range(2, 40, 1)) + list(range(41, 81, 1)) + list(range(82, 119, 1)) + list(range(120, 158, 1)) + \
                list(range(159, 200, 1)) + list(range(201, 239, 1)) + list(range(240, 280, 1)) + list(range(281, 318, 1)) + \
                list(range(319, 357, 1)) + list(range(358, 396, 1)) + list(range(397, 435, 1)) + list(range(436, 472, 1)) + \
                list(range(473, 512, 1)) + list(range(513, 551, 1)) + list(range(551, 590, 1)) + list(range(590, 629, 1)) + \
                list(range(629, 668, 1)) + list(range(668, 706, 1)) + list(range(706, 743, 1)) + list(range(744, 781, 1)) + \
                list(range(784, 823, 1)) + list(range(860, 900, 1)) + list(range(900, 938, 1)) + list(range(1131, 1169, 1)) + \
                list(range(1269, 1307, 1)) + list(range(1572, 1610, 1)) + list(range(3260, 3298, 1)) + \
                list(range(4291, 4329, 1)) + list(range(5664, 5702, 1)) + list(range(6662, 6700, 1))
start_time = time.time()

# create csv file
f = open('pl_game_outcomes.csv', 'w', newline='')
writer = csv.writer(f)
header = ["Team", "Opponent", "Team Score", "Opponent Score", "Score Difference", \
          "Home Game", "Team Yellow Cards", "Team Red Cards", "Opponent Yellow Cards", "Opponent Red Cards", "Outcome"]
writer.writerow(header)

# loop to iterate over all matchday pages and records scores for all teams
for matchday_id in matchday_ids:
    
    # pull page with beautifulsoup
    url = 'https://www.premierleague.com/matchweek/' + str(matchday_id) + '/blog?match=true'
    link = requests.get(url)
    soup = BeautifulSoup(link.content, 'html.parser')
    
    # parse relevant data and calculate additional metrics
    matchday_game_data = soup.find_all('a', attrs = {'class':'matchAbridged fullTime matchWeekMatchContainer'})
    for game in range(len(matchday_game_data)):
        team = matchday_game_data[game].find_all('abbr')[0]['title']
        opponent = matchday_game_data[game].find_all('abbr')[1]['title']
        team_score = int(matchday_game_data[game].find('span', attrs = {'class':'score'}).text[0])
        opponent_score = int(matchday_game_data[game].find('span', attrs = {'class':'score'}).text[2])
        difference = np.abs(team_score - opponent_score)
        team_outcome = (lambda x,y : 'win' if (x-y > 0) else ('loss' if (x-y <0) else 'draw'))(team_score,opponent_score)
        opponent_outcome = (lambda x : 'draw' if (x=='draw') else ('loss' if (x=='win') else 'win'))(team_outcome)
        
        # code to get date and booking information
        teams = [team, opponent]
        events_data = soup.find_all('li', attrs = {'data-ui-modal':'#matchWeekEventsModal'}) # get event data for all events in matchday
        home_away_data = [['', '', ''], ['', '', '']]
        yellows = 0
        reds = 0
        location = 'home'

        # loop that goes through all events to get matchdate and events
        for index, t in enumerate(teams):
            yellows = 0
            reds = 0
            for i in range(len(events_data)):
                event_dict = json.loads(events_data[i]['data-ui-args'])
                event_type = events_data[i].find('span', attrs={'class':'type'}).text.strip()
                if event_dict[f'{location}Team']['name'] != t:
                    continue
                else:
                    #date = event_dict['time']['label'][:-7]
                    if (event_dict[f'{location}Team']['name'] == t) & (event_dict['teamId'] == event_dict[f'{location}Team']['id']):
                        yellows += 1 if event_type == 'Yellow Card' else 0
                        reds += 1 if event_type == 'Red Card' else 0

            location = 'away'
            home_away_data[index] = [yellows, reds]
            
            home_yellows = home_away_data[0][0]
            home_reds = home_away_data[0][1]
            away_yellows = home_away_data[1][0]
            away_reds = home_away_data[0][1]
        
        # add game data to csv
        writer.writerows([[team, opponent, team_score, opponent_score, difference, 1, home_yellows, home_reds, away_yellows, away_reds, team_outcome]])

f.close()
# calculate execution time
ex_time = lambda x : print('Execution Time:', round(x/60, 2), 'minutes') if (x > 60) else print('Execution Time:', round(x, 2), 'seconds')
ex_time(time.time()-start_time)

Execution Time: 25.37 minutes


In [180]:
games = pd.read_csv('pl_game_outcomes.csv')
games.head()

,Team,Opponent,Team Score,Opponent Score,Score Difference,Home Game,Team Yellow Cards,Team Red Cards,Opponent Yellow Cards,Opponent Red Cards,Outcome
0,Arsenal,Norwich City,2,4,2,1,0,0,1,0,loss
1,Chelsea,Oldham Athletic,1,1,0,1,1,0,1,0,draw
2,Coventry City,Middlesbrough,2,1,1,1,0,0,0,0,win
3,Crystal Palace,Blackburn Rovers,3,3,0,1,0,0,0,0,draw
4,Everton,Sheffield Wednesday,1,1,0,1,0,0,0,0,draw


## Calculating Rating Metric

Now that match data has been scraped, I can add an additional feature: team ratings. This is formulated on a structure closely related to the Elo rating system (used in another professional environment - [chess](https://medium.com/purple-theory/what-is-elo-rating-c4eb7a9061e0)). The basics of how this approach works are as follows:
- All teams begin with the same rating.
- At the end of each game, ratings are updated.
- The extent to which ratings are changed depends on the matchup itself. For example:
    - An "underdog" playing who wins against a much higher rated team gets a large increase in rating, while an expected victory will only get the higher rated team a few points in increase.
In this implementation, the traditional Elo formulation is used, with a bit of fine-tuning for the severity of increases/decreases after each match.

Once this is completed, we can see the leaderboard of all soccer teams and their respective ratings.

In [184]:
start_time = time.time()
games['Team Rating'] = 500
games['Opponent Rating'] = 500

for i, team in enumerate(games['Team']): # for every row in dataframe
    team_index = list(games.loc[games['Team']==team].index) #make a list of indices that have that team name
    if i == team_index[0]: # if i is the index corresponding to the first occurence of that team, skip to the next one
        i = team_index[1]

    prev_index = team_index.index(i) - 1 # the index of the previous occurence of that team
    prev_tr = games.iloc[team_index[prev_index]]['Team Rating']
    prev_or = games.iloc[team_index[prev_index]]['Opponent Rating']
        
    # expected outcome (0 for loss, 0.5 for draw, 1 for win)
    team_exp_outcome = 1 / (1+10**((prev_or-prev_tr)/400))
    op_exp_outcome = 1-team_exp_outcome
        
    # compare with actual outcome
    team_act_outcome = [1 if games.iloc[team_index[prev_index]]['Outcome'] == 'win' \
                    else (0.5 if games.iloc[team_index[prev_index]]['Outcome'] == 'draw' \
                    else 0)]
    op_act_outcome = [0 if games.iloc[team_index[prev_index]]['Outcome'] == 'win' \
                    else (0.5 if games.iloc[team_index[prev_index]]['Outcome'] == 'draw' \
                    else 1)]
        
    # make new rating based on player's existing level
    old_ratings = [(prev_tr, team_exp_outcome, team_act_outcome), (prev_or, op_exp_outcome, op_act_outcome)]
    new_ratings = []
        
    for prev, exp, act in old_ratings:
        if prev < 300:
            new_ratings.append(max(prev + 32*(act - exp), 100))
        elif prev > 500:
            new_ratings.append(max(prev + 16*(act - exp), 100))
        else:
            new_ratings.append(max(prev + 24*(act - exp), 100))
        
    games.loc[i, 'Team Rating'] = new_ratings[0]
    games.loc[i, 'Opponent Rating'] = new_ratings[1]
    
ex_time = lambda x : print('Execution Time:', round(x/60, 2), 'minutes') if (x > 60) else print('Execution Time:', round(x, 2), 'seconds')
ex_time(time.time()-start_time)

# get latest ratings
team_list = list(games['Team'].unique())
latest_ratings = {}

for team in team_list:
    latest_index = max(games['Team Rating'].loc[games['Team']==team].index)
    latest_rating = games.loc[latest_index, 'Team Rating']
    latest_ratings[team] = latest_rating
    
sort_ratings = sorted(latest_ratings.items(), key=lambda x: x[1], reverse=True)

# show leaderboard
leaderboard = pd.DataFrame(sort_ratings, columns=['Team', 'Power Index'])
leaderboard.head(10)

Execution Time: 18.58 seconds


,Team,Power Index
0,Manchester City,646.151458
1,Liverpool,614.035279
2,Tottenham Hotspur,594.368099
3,Leicester City,576.612326
4,West Ham United,572.639647
5,Everton,570.036302
6,Arsenal,567.935354
7,Manchester United,561.076200
8,Chelsea,557.039549
9,Newcastle United,547.236702


## Base Joint Probability (probabilities based off of entire dataset)

Now that we've got all the building blocks, we're ready to start creating our models! I would like to start out by explaining the underlying goal. Instead of predicting an end result right off the bat ("win", "loss", or "draw"), we must try to calculate the probabilities of scoring a certain number of goals.

FiveThirtyEight has a wonderful explanation of their methodology, [which uses the same approach](https://fivethirtyeight.com/methodology/how-our-club-soccer-predictions-work/). To boil it down, think of a single game. For each team, there is a certain probability of scoring one goal, two goals, three goals, and so on. It is the same for the other team. By calculating joint probabilities, we can get the chances of every single outcome available for a given game. It is only a matter of adding these probabilities up for all "wins", "losses", and "draws" to determine final probabilities.

A useful visualization that details this process is below:

<div>
<img src="prob_matrix.png" width="500"/>
</div>

<i><center>*Figure 2: an example of a probability matrix for a given matchup*<center></i>

The first model created takes inspiration from this notion. However, the approach is much simpler. For a given matchup of Team A and Team B, probabilities for each team are calculated by looking at the fraction of that team's goals that coincide with that value. 

For example, to find Team A's probability of scoring one goal, we find the number of games in which Team A scored one goal and divide that by the total number of games played by that team. Once these are calculated, we define a probability matrix (as in Figure 2), add our areas, and we are left with probabilities for winning, losing or drawing!

In [175]:
def base_predict_outcome(team, opponent):
    team_counts = games.loc[(games['Team']==team), 'Team Score'].value_counts(normalize=True).sort_index()
    opponent_counts = games.loc[(games['Team']==opponent), 'Team Score'].value_counts(normalize=True).sort_index()

    if len(team_counts) > len(opponent_counts):
        opponent_counts = np.pad(opponent_counts, (0, len(team_counts) - len(opponent_counts)))
    elif len(team_counts) < len(opponent_counts):
        team_counts = np.pad(team_counts, (0, len(opponent_counts) - len(team_counts)))
    team_prob = np.matrix(team_counts)
    op_prob = np.matrix(opponent_counts)

    A = np.matmul(op_prob.T, team_prob)
    team_prob = round((np.triu(A).sum()-np.trace(A)), 2)
    opponent_prob = round((np.tril(A).sum()-np.trace(A)), 2)
    draw_prob = round(np.trace(A), 2)
    projected_outcome = team if (opponent_prob <= team_prob >= draw_prob) else \
                        (opponent if (team_prob <= opponent_prob >= draw_prob) else 'Draw')
    
    #print(f'{team} win: {team_prob}\n{opponent} win: {opponent_prob}\nDraw: {draw_prob}')
    return projected_outcome

We may test this simple probabilistic model on the entire dataset and get an overall accuracy:

In [186]:
# for each row
correct = 0
for i in list(range(len(games))):
    # get team and opponent names
    team = games['Team'][i]
    opponent = games['Opponent'][i]
    
    # run function to find projected outcome
    projected_outcome = base_predict_outcome(team, opponent)
    if (projected_outcome == team) & (games['Outcome'][i] == 'win'):
        correct += 1
    elif (projected_outcome == opponent) & (games['Outcome'][i] == 'loss'):
        correct += 1
    elif (projected_outcome == 'draw') & (games['Outcome'][i] == 'draw'):
        correct += 1

# get test accuracy
base_acc = correct/len(games)
print(f'Base Model Accuracy = {round(base_acc*100,2)}%')

Base Model Accuracy = 48.16%


We see that we have achieved an overall accuracy rating of 48.16%, which is already within our threshold!

## Multinomial Logistic Regression Method

Now that we have a probabilistic model to compare against, we can start working on a logistic regression implementation. As with the previous section, we are trying to obtain probabilities of each score outcome. We are specifically using multinomial logistic regression, a generalization of logistic regression to multiple discrete outcomes. This type of model was chosen specifically due to the fact that results are directly interpretable as probabilities, making it easier for us to get to our end result.

In [192]:
games['Score Difference'] = games['Team Score'] - games['Opponent Score']
games2 = np.array(games[['Team Rating', 'Opponent Rating']])

logit = games2[399:10000,:]
scaler = StandardScaler().fit(logit)
logit = scaler.transform(logit)
# make dummy variables for team names and concat
#logit_df = pd.concat([games[['Home Game', 'Team Rating', 'Opponent Rating']], teams_dummies, opponents_dummies], axis=1)
logit_df = logit

# Logistic Regression
clf = LogisticRegression(multi_class='multinomial', random_state=0, max_iter=5000)

X = logit_df
y_team = games[f'Team Score'].iloc[399:10000]
y_opp = games[f'Opponent Score'].iloc[399:10000]

# fit and test models to predict goals scored for each side (both team and ops)
for i in [y_team, y_opp]:
    
    # set cross-validation parameters and run k-fold cross validation
    kf = KFold(n_splits = 10, shuffle=True, random_state = 47)
    cv = cross_val_score(clf, X, i, cv = kf)
    result = np.mean(cv)
    
# If model cv accuracy is acceptable, we can train on full dataset and predict future matchups
team_clf = LogisticRegression(random_state=0, max_iter=5000).fit(X, y_team)
opp_clf = LogisticRegression(random_state=0, max_iter=5000).fit(X, y_opp)

# get probabilities for home and away scores using "test set" of last 3,000 or so rows of full dataset
logit_test = games2[10000:,:]
logit_data = logit_test
y_team = games[f'Team Score'].iloc[10000:].tolist()
y_opp = games[f'Opponent Score'].iloc[10000:].tolist()
actual_outcome = games['Outcome'].iloc[10000:].tolist()
scaler_transform = scaler.transform(logit_data)
team_probs = team_clf.predict_proba(scaler_transform)
opp_probs = opp_clf.predict_proba(scaler_transform)

predicted_outcomes = []
correct = 0
# change games column values to win
for i in list(range(len(logit_test))):
    # obtain joint probability matrix 
    team_prob_matrix = np.matrix(team_probs[i])
    op_prob_matrix = np.matrix(opp_probs[i])
    A = np.matmul(op_prob_matrix.T, team_prob_matrix)
    #plt.imshow(A, interpolation='none')
    #plt.show()

    # calculate outcomes for win, loss, and draw
    team_prob = round((np.triu(A).sum()-np.trace(A)), 2)
    opponent_prob = round((np.tril(A).sum()-np.trace(A)), 2)
    draw_prob = round(np.trace(A), 2)
    projected_outcome = 'win' if (opponent_prob <= team_prob >= draw_prob) else \
                        ('loss' if (team_prob <= opponent_prob >= draw_prob) else 'draw')
    predicted_outcomes.append(projected_outcome)
    if predicted_outcomes[i] == actual_outcome[i]:
        correct += 1

# get test accuracy
test_acc = correct/len(logit_test)
print(f'Test Accuracy (Logistic Regression) = {round(test_acc*100,2)}%')

Test Accuracy (Logistic Regression) = 47.72%


Using only two features, we have come up with a model with relatively the same amount of accuracy as our base model. Both are within the threshold we were looking for, which is great!

In [168]:
def logit_predict_outcome(team, opponent):
    # find last row occurence of team, opponent to find team rating, opponent rating
    team_home_list = games.index[games['Team'] == team].tolist()
    team_away_list = games.index[games['Opponent'] == team].tolist()
    if max(team_home_list) > max(team_away_list):
        team_rating = games['Team Rating'][max(team_home_list)]
    else:
        team_rating = games['Opponent Rating'][max(team_away_list)]
        
    opp_home_list = games.index[games['Team'] == opponent].tolist()
    opp_away_list = games.index[games['Opponent'] == opponent].tolist()
    if max(opp_home_list) > max(opp_away_list):
        opp_rating = games['Team Rating'][max(opp_home_list)]
    else:
        opp_rating = games['Opponent Rating'][max(opp_away_list)]
        
    logit_data = np.array([[team_rating, opp_rating]])
    scaler_transform = scaler.transform(logit_data)
    team_probs = team_clf.predict_proba(scaler_transform)
    opp_probs = opp_clf.predict_proba(scaler_transform)
    # find probabilities with rating inputs
    
    # obtain joint probability matrix 
    team_prob_matrix = np.matrix(team_probs)
    op_prob_matrix = np.matrix(opp_probs)
    A = np.matmul(op_prob_matrix.T, team_prob_matrix)
    #plt.imshow(A, interpolation='none')
    #plt.show()

    # calculate outcomes for win, loss, and draw
    team_prob = round((np.triu(A).sum()-np.trace(A)), 2)
    opponent_prob = round((np.tril(A).sum()-np.trace(A)), 2)
    draw_prob = round(np.trace(A), 2)
    projected_winner = team if (opponent_prob <= team_prob >= draw_prob) else \
                        (opponent if (team_prob <= opponent_prob >= draw_prob) else 'draw')
    
    #print(f"{team} Win: {team_prob}\n{opponent} Win: {opponent_prob}\nDraw: {draw_prob}")
    return projected_winner

## Comparing Models on 2022 Matchweek 1

The final step in this project is to compare results of our models with FiveThirtyEight's prediction for a new matchweek - in our case, the first week of the Premier League's 2022/23 season. Let's see how we do!

In [195]:
matchweek_df = pd.DataFrame(columns=["Team", "Opponent", "Base Prediction", "Logistic Prediction", "FiveThirtyEight Prediction", "Actual Result"])
teams = ["Crystal Palace", "Everton", "Tottenham Hotspur", "Newcastle United", "Leeds United", "Bournemouth", "Fulham", "West Ham United", "Manchester United", "Leicester City"]
opponents = ["Arsenal", "Chelsea", "Southampton", "Nottingham Forest", "Wolverhampton Wanderers", "Aston Villa", "Liverpool", "Manchester City", "Brighton and Hove Albion", "Brentford"]
five_thirty_eight = ["Arsenal", "Chelsea", "Tottenham Hotspur", "Newcastle United", "Leeds United", "Bournemouth", "Liverpool", "Manchester City", "Manchester United", "Leicester City"]
actual_results = ["Arsenal", "Chelsea", "Tottenham Hotspur", "Newcastle United", "Leeds United", "Bournemouth", "Draw", "Manchester City", "Brighton and Hove Albion", "Draw"]

matchweek_df['Team'] = teams
matchweek_df['Opponent'] = opponents
matchweek_df['FiveThirtyEight Prediction'] = five_thirty_eight
matchweek_df['Actual Result'] = actual_results

for i in list(range(len(matchweek_df))):
    team = matchweek_df['Team'][i]
    opponent = matchweek_df['Opponent'][i]
    
    # update df
    matchweek_df['Base Prediction'][i] = base_predict_outcome(team, opponent)
    matchweek_df['Logistic Prediction'][i] = logit_predict_outcome(team, opponent)
    
matchweek_df    

,Team,Opponent,Base Prediction,Logistic Prediction,FiveThirtyEight Prediction,Actual Result
0,Crystal Palace,Arsenal,Arsenal,Arsenal,Arsenal,Arsenal
1,Everton,Chelsea,Chelsea,Chelsea,Chelsea,Chelsea
2,Tottenham Hotspur,Southampton,Tottenham Hotspur,Tottenham Hotspur,Tottenham Hotspur,Tottenham Hotspur
3,Newcastle United,Nottingham Forest,Newcastle United,Newcastle United,Newcastle United,Newcastle United
4,Leeds United,Wolverhampton Wanderers,Leeds United,Leeds United,Leeds United,Leeds United
5,Bournemouth,Aston Villa,Bournemouth,Bournemouth,Bournemouth,Bournemouth
6,Fulham,Liverpool,Liverpool,Liverpool,Liverpool,Draw
7,West Ham United,Manchester City,Manchester City,Manchester City,Manchester City,Manchester City
8,Manchester United,Brighton and Hove Albion,Manchester United,Manchester United,Manchester United,Brighton and Hove Albion
9,Leicester City,Brentford,Leicester City,Leicester City,Leicester City,Draw


In [196]:
base_acc = sum(matchweek_df['Base Prediction'] == matchweek_df['Actual Result'])/10
logit_acc = sum(matchweek_df['Logistic Prediction'] == matchweek_df['Actual Result'])/10
fivethirtyeight_acc = sum(matchweek_df['FiveThirtyEight Prediction'] == matchweek_df['Actual Result'])/10

print(f'Base Accuracy = {base_acc}\nLogistic Regression Accuracy = {logit_acc}\nFiveThirtyEight Accuracy = {fivethirtyeight_acc}')

Base Accuracy = 0.7
Logistic Regression Accuracy = 0.7
FiveThirtyEight Accuracy = 0.7


Fantastic! Over the course of the first ten games, we've matched accuracy with FiveThirtyEight's model, with all accuracy rates reaching 70%.

### Summary and Next Steps

Over the course of this project, we have web scraped data and created two separate predictive models to forecast soccer game outcomes. We found that the models we crafted were virtually similar in accuracy, and both accuracy rates fell very close to 50%. This was within the range we were hoping to achieve.

Where could we have improved? While web scraping opens the doors to many new sources, sometimes those sources may be limiting in the information they have. Dynamic web pages, which are becoming more common, inhibit simple web scrapers from obtaining all page information from simply loading the specific URL. I found that there were many chances to get further game information that would require this dynamic element. In the future, we could implement a more complex web scraping implementation to get more features.

Due to this data limitation, we only fit our logistic regression model with two features (the team ratings). It is almost certain that with an increase in useful features, we would see an accuracy rate surpassing 50%.

How might we expand on the findings of this project? Although the bulk of data regarding games was found, there is new information coming through every week. We could potentially set up our code to cyclically scrape each matchweek's data and retraining our models based on the addition of that new information. The automation of this process could even culminate with alerts sent to an individual's device with matchweek predictions!